In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import random

def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_torch(seed=666)

In [2]:
TEST_PATH = '../input/ranzcr-clip-catheter-line-classification/test'
FOLDS_PATH = '../input/split-k-folds/'
TRAIN_PATH = '../input/ranzcr-clip-catheter-line-classification/train'

target_size=11
target_cols=['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal',
             'NGT - Abnormal', 'NGT - Borderline', 'NGT - Incompletely Imaged', 'NGT - Normal', 
             'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal',
             'Swan Ganz Catheter Present']

# cargamos los DATOS
dfx = pd.read_csv(FOLDS_PATH + "train_folds.csv") 
 
# filtramos. Por defecto dejo en "0" el fold_test
df_test_fold = dfx[dfx.fold_test == 0].reset_index(drop=True)
print(df_test_fold.shape)
df_test_fold['file_path'] = df_test_fold.StudyInstanceUID.apply(lambda x: os.path.join('../input/ranzcr-clip-catheter-line-classification/train', f'{x}.jpg'))
df_test_fold.head()

(1245, 15)


,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present,PatientID,fold,fold_test,file_path
0,1.2.826.0.1.3680043.8.498.68286643202323212801...,0,0,0,0,0,0,0,1,0,0,0,c31019814,0,0,../input/ranzcr-clip-catheter-line-classificat...
1,1.2.826.0.1.3680043.8.498.10050203009225938259...,0,0,0,0,0,0,0,0,0,1,0,207685cd1,0,0,../input/ranzcr-clip-catheter-line-classificat...
2,1.2.826.0.1.3680043.8.498.23522123195677570717...,0,0,0,0,0,0,0,1,0,0,0,20335f7d1,4,0,../input/ranzcr-clip-catheter-line-classificat...
3,1.2.826.0.1.3680043.8.498.96303608016739892857...,0,0,0,0,0,0,0,1,0,0,0,528f079f0,2,0,../input/ranzcr-clip-catheter-line-classificat...
4,1.2.826.0.1.3680043.8.498.30798392240062365831...,0,0,0,0,0,0,0,0,0,1,0,3c1fc81a3,2,0,../input/ranzcr-clip-catheter-line-classificat...


In [3]:
model_names = ['fold0', 'fold1', 'fold2', 'fold3', 'fold4'] * 8
enet_type  = [ 
#     'efficientB5_512', 'efficientB5_512', 'efficientB5_512', 'efficientB5_512', 'efficientB5_512', 
    'effficientB4-attention', 'effficientB4-attention', 'effficientB4-attention', 'effficientB4-attention', 'effficientB4-attention',
    'resnext50_32x4d', 'resnext50_32x4d', 'resnext50_32x4d', 'resnext50_32x4d', 'resnext50_32x4d',
    'resnet200d_320', 'resnet200d_320', 'resnet200d_320', 'resnet200d_320', 'resnet200d_320',
    'resnet200d_320_512', 'resnet200d_320_512', 'resnet200d_320_512', 'resnet200d_320_512', 'resnet200d_320_512',
#     'resnet200d_Mejores', 'resnet200d_Mejores', 'resnet200d_Mejores', 'resnet200d_Mejores', 'resnet200d_Mejores',
    'inceptionV2_640', 'inceptionV2_640', 'inceptionV2_640', 'inceptionV2_640', 'inceptionV2_640',
    'efficient-B7', 'efficient-B7', 'efficient-B7', 'efficient-B7', 'efficient-B7',
    'xception_840','xception_840','xception_840','xception_840','xception_840',
    'efficientnet-B5', 'efficientnet-B5', 'efficientnet-B5', 'efficientnet-B5', 'efficientnet-B5'
]

In [4]:
# model_names = ['fold0', 'fold1', 'fold2', 'fold3', 'fold4']

# enet_type  = ['resnet200d_320', 'resnet200d_320', 'resnet200d_320', 'resnet200d_320', 'resnet200d_320']

# model_path = [
#     '../input/resnet200d-640/resnet200d_320_fold0_step0_best_loss_cpu.pth',
#         '../input/resnet200d-640/resnet200d_320_fold1_step0_best_loss_cpu.pth',
#         '../input/resnet200d-640/resnet200d_320_fold2_step0_best_loss_cpu.pth',
#         '../input/resnet200d-640/resnet200d_320_fold3_step0_best_loss_cpu.pth',
#         '../input/resnet200d-640/resnet200d_320_fold4_step0_best_loss_cpu.pth'
# ]

In [5]:
# calculando pesos a partir del score
def calculate_weigths_inverse(lista_scores):
    
    lista_scores = np.array(lista_scores)

    weights = 100 - lista_scores
    total_score = np.sum(weights)

    weights = weights/total_score

    return weights

Guardo todo en una lista

In [6]:
input_folds = []
for i in range(len(enet_type)):
    preds = pd.read_csv('../input/buscador-de-pesos-ranzcr/' + enet_type[i] + '_' + model_names[i] + '.csv')
    input_folds.append(preds)

In [7]:
input_folds[0][['ETT - Normal']]

,ETT - Normal
0,0.000324
1,0.000206
2,0.000030
3,0.000411
4,0.005040
...,...
1240,0.000364
1241,0.000077
1242,0.000219
1243,0.715672


Creo un modelo por cada label y, luego, calculo la media de todos para asignar el peso final

In [8]:
def get_coeficients_normalized(coef):
    suma = np.sum(coef)
    return coef / suma

def save_csv(target, coef, names):
    df_model_coef = pd.DataFrame(
    {'name': names,
     'weigth': coef
    })
    df_model_coef.to_csv(target + '_weigths.csv', index=False)
    df_model_coef

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

def get_weigths(len_enet_type_input, input_folds, names, verbose = 1):
    coeficientes = []
    for i in range(len(target_cols)):
        # voy columna/target por columna
        target = target_cols[i]
        print('Searching weigths for ' + target)
        # pillo las variables 
        cols_for_model = []
        for j in range(len_enet_type_input):
            col = input_folds[j][[target]]
            # lo paso a 0's o 1's
    #         col = (col >= 0.5).astype(int)
            cols_for_model.append(col)
        # creo el modelo
        X = np.column_stack(cols_for_model)
        y = df_test_fold[[target]].values.ravel()
        # ---------añadido-------------
        X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.1, random_state=66)
        # --------------------------
        reg = GradientBoostingClassifier().fit(X_train, y_train)
        print('Score obtenido: ' + str(round(reg.score(X_test, y_test), 2)))
        reg_coef = reg.feature_importances_
        best_model_index = np.argmax(reg_coef)
        worst_model_index = np.argmin(reg_coef)
        if verbose == 1:
            print('El modelo con más peso es: ' + enet_type[best_model_index] + '_' + model_names[best_model_index])
            print('El modelo con menos peso es: ' + enet_type[worst_model_index] + '_' + model_names[worst_model_index])
            print(reg_coef)
            print(np.sum(reg_coef))
        coeficientes.append(reg_coef) # coef_
    #     coef_normalized = get_coeficients_normalized(reg_coef.ravel()) # solo en caso de no ser randomforest
        save_csv(target, reg_coef, names)
        print('---------------------------------------------------------------------------')
        
    return coeficientes

In [10]:
len_enet_type = len(enet_type)
names = [a + '_' + b for a, b in zip(enet_type, model_names)]
coeficientes = get_weigths(len_enet_type, input_folds, names)
print('Suma coeficientes: ' + str(np.sum(coeficientes)))


Searching weigths for ETT - Abnormal
Score obtenido: 0.98
El modelo con más peso es: resnet200d_320_512_fold2
El modelo con menos peso es: inceptionV2_640_fold2
[4.75602228e-03 5.31455473e-08 2.31335915e-05 6.94589048e-11
 6.93296724e-11 1.55109586e-03 2.41921228e-04 3.81893418e-05
 2.79663080e-04 1.15403485e-03 7.65664856e-10 1.75725267e-07
 8.48373336e-11 2.93806769e-04 6.21142527e-03 1.87705535e-04
 1.18863873e-05 6.57470519e-01 8.60292571e-03 6.51317823e-02
 6.71610693e-04 1.20561931e-07 5.68681206e-11 3.40794545e-06
 1.71667612e-03 8.67981998e-04 1.31920394e-04 4.18987690e-04
 4.59947260e-05 2.49422349e-04 1.46390198e-03 8.11113945e-03
 5.23169679e-03 1.42918904e-03 5.08525278e-06 8.05856874e-02
 4.26870122e-03 3.42287086e-08 1.38322887e-01 1.05212138e-02]
1.0000000000000002
---------------------------------------------------------------------------
Searching weigths for ETT - Borderline
Score obtenido: 0.95
El modelo con más peso es: resnet200d_320_512_fold2
El modelo con menos p

In [11]:
# aqui hago la media de coeficientes y tiro palante
final_coefs = np.mean(coeficientes, axis=0).ravel()
best_model_index = np.argmax(final_coefs)
worst_model_index = np.argmin(final_coefs)
print('El modelo con más peso es: ' + enet_type[best_model_index] + '_' + model_names[best_model_index])
print('El modelo con menos peso es: ' + enet_type[worst_model_index] + '_' + model_names[worst_model_index])
print(final_coefs)

El modelo con más peso es: resnet200d_320_512_fold2
El modelo con menos peso es: resnext50_32x4d_fold0
[0.01165726 0.00502629 0.0099379  0.00395564 0.00743968 0.00333874
 0.0138534  0.0064636  0.01085462 0.00724344 0.00349924 0.0277735
 0.01398341 0.06908749 0.02002097 0.12686702 0.11572889 0.12879818
 0.10237584 0.03589793 0.00487336 0.01312275 0.00391434 0.00363623
 0.00899499 0.06735711 0.00409518 0.0111208  0.02152699 0.01735415
 0.01560037 0.00854909 0.02003217 0.00415241 0.00770603 0.01544626
 0.01247846 0.01024043 0.01892994 0.0070659 ]


In [12]:
# guardo
# save in a csv
names = [a + '_' + b for a, b in zip(enet_type, model_names)]
df_model_coef = pd.DataFrame(
    {'name': names,
     'weigth': final_coefs,
     'enet_type': enet_type
    })
df_model_coef.to_csv('models_weigths.csv', index=False)
df_model_coef.sort_values(by=['weigth'], ascending=False)

,name,weigth,enet_type
17,resnet200d_320_512_fold2,0.128798,resnet200d_320_512
15,resnet200d_320_512_fold0,0.126867,resnet200d_320_512
16,resnet200d_320_512_fold1,0.115729,resnet200d_320_512
18,resnet200d_320_512_fold3,0.102376,resnet200d_320_512
13,resnet200d_320_fold3,0.069087,resnet200d_320
25,efficient-B7_fold0,0.067357,efficient-B7
19,resnet200d_320_512_fold4,0.035898,resnet200d_320_512
11,resnet200d_320_fold1,0.027774,resnet200d_320
28,efficient-B7_fold3,0.021527,efficient-B7
32,xception_840_fold2,0.020032,xception_840


ACCURACIES

In [13]:
# funcioens un poco cambiadas para este notebook
def assign_weights_by_target(preds, len_enet_type):
    final_preds = np.zeros((preds[0].shape[0], 11))
    # recorro las columnas
    for i in range(len(target_cols)):
        target = target_cols[i]
        target_w = pd.read_csv('./' + target + '_weigths.csv').values
        # recorro los modelos
        for j in range(len_enet_type):
            # preds por target
            model_preds = preds[j]
            model_preds_target = model_preds[:, i]
            # weigth
            weigth = target_w[j, 1]
            model_preds_target = model_preds_target * weigth
            final_preds[:, i] += model_preds_target
    
    return final_preds

def assign_weigths(preds, len_enet_type):
    final_preds = np.zeros((preds[0].shape[0], 11))
    weigths = final_coefs
    for j in range(len_enet_type):
        model_preds = preds[j]
        model_preds = model_preds * weigths[j]
        final_preds += model_preds
        
    return final_preds

In [14]:
# pasa de pandas a coger las columnas target directamente en formato numpy
def get_preds_by_name(names_input):
    test_preds = []
    for i in range(len(names_input)):
        preds = pd.read_csv('../input/buscador-de-pesos-ranzcr/' + names_input[i] + '.csv')
        preds = preds.iloc[:, 1:12].values
        test_preds.append(preds)   
        
    return test_preds

In [15]:
# obtengo las predicciones de los modelos
names = sum_list = [a + '_' + b for a, b in zip(enet_type, model_names)]
test_preds = get_preds_by_name(names)

In [16]:
from sklearn.metrics import roc_auc_score

def get_auc(y_pred, y_real):
    acc_fold = np.mean([roc_auc_score(y_real[:, i], y_pred[:, i]) for i in range(11)])

    print("Final ACC for this fold:", acc_fold)
    
y_real = df_test_fold.iloc[:, 1:12].values
    
# media
get_auc(np.mean(test_preds, axis = 0), y_real)

# pesos generales
y_preds = assign_weigths(test_preds, len(enet_type))
get_auc(y_preds, y_real)

# pesos por target
y_preds = assign_weights_by_target(test_preds, len(enet_type))
get_auc(y_preds, y_real)

Final ACC for this fold: 0.9488993965393466
Final ACC for this fold: 0.9569646637449981
Final ACC for this fold: 0.9572801837083499


**PRIMERA CRIBA: quito los que tengan un peso inferior a la media de weigths**
---

In [17]:
primera_criba_names = df_model_coef[df_model_coef['weigth']>df_model_coef['weigth'].quantile(.75)] # quantil 0.7 sin chest y mejores
print(primera_criba_names.sort_values(by=['weigth'], ascending=False))

input_folds = []
for i in range(len(primera_criba_names.name)):
    preds = pd.read_csv('../input/buscador-de-pesos-ranzcr/' + primera_criba_names.iloc[i, 0] + '.csv')
    input_folds.append(preds)
    
len_enet_type = len(primera_criba_names.name)
coeficientes = get_weigths(len_enet_type, input_folds, primera_criba_names.name, verbose = 0)
print('Suma coeficientes: ' + str(np.sum(coeficientes)))

                        name    weigth           enet_type
17  resnet200d_320_512_fold2  0.128798  resnet200d_320_512
15  resnet200d_320_512_fold0  0.126867  resnet200d_320_512
16  resnet200d_320_512_fold1  0.115729  resnet200d_320_512
18  resnet200d_320_512_fold3  0.102376  resnet200d_320_512
13      resnet200d_320_fold3  0.069087      resnet200d_320
25        efficient-B7_fold0  0.067357        efficient-B7
19  resnet200d_320_512_fold4  0.035898  resnet200d_320_512
11      resnet200d_320_fold1  0.027774      resnet200d_320
28        efficient-B7_fold3  0.021527        efficient-B7
32        xception_840_fold2  0.020032        xception_840
Searching weigths for ETT - Abnormal
Score obtenido: 0.98
---------------------------------------------------------------------------
Searching weigths for ETT - Borderline
Score obtenido: 0.96
---------------------------------------------------------------------------
Searching weigths for ETT - Normal
Score obtenido: 0.96
-------------------------

In [18]:
# aqui hago la media de coeficientes y tiro palante
final_coefs = np.mean(coeficientes, axis=0).ravel()
best_model_index = np.argmax(final_coefs)
worst_model_index = np.argmin(final_coefs)
print('El modelo con más peso es: ' + primera_criba_names.name.values[best_model_index])
print('El modelo con menos peso es: ' + primera_criba_names.name.values[worst_model_index])
print(final_coefs)

# guardo
# save in a csv
df_model_coef = pd.DataFrame(
    {'name': primera_criba_names.name.values,
     'weigth': final_coefs
    })
df_model_coef.to_csv('models_weigths_criba.csv', index=False)
df_model_coef.sort_values(by=['weigth'], ascending=False)

El modelo con más peso es: resnet200d_320_512_fold0
El modelo con menos peso es: resnet200d_320_fold1
[0.04026618 0.08973347 0.17344747 0.15401373 0.11175503 0.15440983
 0.08071557 0.064087   0.07614562 0.05542609]


,name,weigth
2,resnet200d_320_512_fold0,0.173447
5,resnet200d_320_512_fold3,0.154410
3,resnet200d_320_512_fold1,0.154014
4,resnet200d_320_512_fold2,0.111755
1,resnet200d_320_fold3,0.089733
6,resnet200d_320_512_fold4,0.080716
8,efficient-B7_fold3,0.076146
7,efficient-B7_fold0,0.064087
9,xception_840_fold2,0.055426
0,resnet200d_320_fold1,0.040266


In [19]:
test_preds_primera_criba = get_preds_by_name(primera_criba_names.name.values)

In [20]:
  # media
get_auc(np.mean(test_preds_primera_criba, axis = 0), y_real)

# pesos generales
y_preds = assign_weigths(test_preds_primera_criba, len(primera_criba_names.name))
get_auc(y_preds, y_real)

# pesos por target
y_preds = assign_weights_by_target(test_preds_primera_criba, len(primera_criba_names.name))
get_auc(y_preds, y_real)

Final ACC for this fold: 0.9566471285296185
Final ACC for this fold: 0.9582880949046949
Final ACC for this fold: 0.9610609791597813
